In [1]:
import lumipy as lm
from lusidjam import RefreshingToken as rt

atlas = lm.get_atlas(token=rt())

# Tutorial 4 - Table Variables and Scalar Variables

Query variables in luminesce are table or scalar valued sub-queries that can be re-used within the query. 

## Table Variables

Table variables are queries that resolve to a table and once built in lumipy they behave like any source table class. They can be joined with other tables or queried back with `select(...)`. The columns are manifested as attributes on the table variable object and they take on the name of their alias (if they're aliased). 

In [2]:
quotes = atlas.lusid_instrument_quote()

In [3]:
table_var = quotes.select(
    quotes.instrument_id
).where(
    quotes.lineage == 'YahooFinance'
).group_by(
    quotes.instrument_id
).aggregate(
    MinVal=quotes.value.min(),
    MedianVal=quotes.value.median(),
    MaxVal=quotes.value.max()    
).to_table_var()

table_var.get_columns()

[└•Text ⬅ SQL Piece: [InstrumentId]
    └•[COLUMN INPUT]
       @tv_487287495033147632: [InstrumentId],
 └•Decimal ⬅ SQL Piece: [MinVal]
    └•[COLUMN INPUT]
       @tv_487287495033147632: MinVal,
 └•Decimal ⬅ SQL Piece: [MedianVal]
    └•[COLUMN INPUT]
       @tv_487287495033147632: MedianVal,
 └•Decimal ⬅ SQL Piece: [MaxVal]
    └•[COLUMN INPUT]
       @tv_487287495033147632: MaxVal]

## Scalar Variables

Scalar variables are subqueries that resolve to a single value (one column value and one row). This scalar value can then be used in various places where you'd use a literal value. 

In [4]:
holding = atlas.lusid_portfolio_holding()

In [5]:
total_cost = holding.select(
    TotalCost=holding.cost_amount_portfolio_currency.sum()
).where(
    holding.portfolio_scope == 'Finbourne-Examples'
).to_scalar_var()

qry = holding.select(
    holding.portfolio_code,
    holding.lusid_instrument_id, 
    holding.cost_amount_portfolio_currency,
    CostFractionPc=100*holding.cost_amount_portfolio_currency/total_cost
).where(
    holding.portfolio_scope == 'Finbourne-Examples'
).order_by(
    (holding.cost_amount_portfolio_currency/total_cost).descending()
).limit(5)

In [6]:
df = qry.go()
df

,PortfolioCode,LusidInstrumentId,CostAmountPortfolioCurrency,CostFractionPc
0,Global-Equity,LUID_JTQY6QFI,1.822715e+09,19.549961
1,Global-Equity,LUID_1W5WYN3O,1.760183e+09,18.879259
2,Global-Equity,LUID_4CINGMZM,1.322168e+09,14.181222
3,Global-Equity,LUID_J4G8D0DV,1.146539e+09,12.297467
4,Global-Equity,LUID_GJ9TICS4,1.050759e+09,11.270165


In [7]:
qry.print_sql()

@@sv_66112660981258678 = 
select
  total([CostAmountPortfolioCurrency]) as [TotalCost] 
from
  Lusid.Portfolio.Holding 
where
  [PortfolioScope] = 'Finbourne-Examples';


select
  [PortfolioCode], [LusidInstrumentId], [CostAmountPortfolioCurrency], ([CostAmountPortfolioCurrency] * 100) / @@sv_66112660981258678 as [CostFractionPc] 
from
  Lusid.Portfolio.Holding 
where
  [PortfolioScope] = 'Finbourne-Examples' 
order by
  [CostAmountPortfolioCurrency] / @@sv_66112660981258678 desc 
limit
  5
